# Quantum Circuits
Quantum computers can only use a specific set of gates (universal gate set). Given the entanglers and their amplitudes found in Step 3, one can find corresponding representation of these operators in terms of elementary gates using the following procedure.

In [7]:

import tequila as tq
from utility import *
import time

In [ ]:
#Running with cell goes over all possible optinons  
molecules=['h2','h4','lih','h2o','n2','nh3']
methods=['hf','cisd','ccsd','fci']
basies=['sto-3g','6-31g']
qubit_transfms=['jw','bk'] 

In [85]:
#Fcus on one molecule
molecules=['h2']
methods=['cisd']
basies=['sto-3g']
qubit_transfms=['jw'] 

qubit_transf = 'jw' # Jordan-Wigner transformations

In [86]:
R=1 #bond_length

for mol in molecules:

    for basis in basies:

        for qubit_transf in qubit_transfms:
                    
            start=time.time()
            
            H = get_qubit_hamiltonian(mol, geometry=R, basis=basis, qubit_transf=qubit_transf)
            print("Take a look at H!")
            
            xyz_data = get_molecular_data(mol, geometry=R, xyz_format=True)
            Htq = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)

            E_cisd=obtain_PES(mol, [R], basis, 'cisd')
            print('cisd'.upper()," ground state energy is:",E_cisd)
            
            n_spin_orbitals=2*Htq.n_orbitals
            n_electrons=Htq.n_electrons
            print("Numbers of electrons:",n_electrons,
                  "\nUsing ",qubit_transf.upper()," in ",basis.upper(),"basis.",
                  "\nNumber of spin-orbitals (qubits):",n_spin_orbitals,
                  "\nNumber of Hamiltonian terms:",len(H.terms)) 

            H_eff=taper_hamiltonian(H, n_spin_orbitals, n_electrons, qubit_transf)
            print("Number of terms in Taper effective Hamiltonian (H_eff):",len(H_eff.terms))

            
            if mol=='h2':
                # Building the matrix representation of the effective Hamiltonian
                I, X, Z = np.identity(2), np.array([[0, 1], [1, 0]]), np.array([[1, 0], [0, -1]])
                H_matrix = -0.53105134 * I + 0.19679058 * X - 0.53505729 * Z

                # Obtain the eigenvalues
                eigvals, _ = np.linalg.eigh(H_matrix)
                print("The eigenvalues of the effective Hamiltonian:{}".format(eigvals))
            else:
                print("H_matrix for qubit-tapering technique not yet implemeneted for ",mol.upper())

            print(mol.upper(),"Hamiltonian using ",qubit_transf.upper()," in basis:",
                  basis.upper(),"generated for:",time.time()-start," sec\n")
            
            

Take a look at H!
r=1.0000, E =  -1.10115 Eh
CISD  ground state energy is: [-1.10115033]
Numbers of electrons: 2 
Using  JW  in  STO-3G basis. 
Number of spin-orbitals (qubits): 4 
Number of Hamiltonian terms: 15
Number of terms in Taper effective Hamiltonian (H_eff): 3
The eigenvalues of the effective Hamiltonian:[-1.10115031  0.03904763]
H2 Hamiltonian using  JW  in basis: STO-3G generated for: 1.2697703838348389  sec



In [87]:
H

-0.32760818967480887 [] +
-0.04919764587136755 [X0 X1 Y2 Y3] +
0.04919764587136755 [X0 Y1 Y2 X3] +
0.04919764587136755 [Y0 X1 X2 Y3] +
-0.04919764587136755 [Y0 Y1 X2 X3] +
0.13716572937099497 [Z0] +
0.15660062488237947 [Z0 Z1] +
0.10622904490856075 [Z0 Z2] +
0.15542669077992832 [Z0 Z3] +
0.13716572937099492 [Z1] +
0.15542669077992832 [Z1 Z2] +
0.10622904490856075 [Z1 Z3] +
-0.13036292057109117 [Z2] +
0.16326768673564346 [Z2 Z3] +
-0.13036292057109117 [Z3]

In [88]:
H_eff

-0.5310513494337641 [] +
0.1967905834854702 [X0] +
-0.5350572998841723 [Z0]

In [89]:
qwc_list = get_qwc_group(H)
print('Fragments 1: \n{}\n'.format(qwc_list[0]))
print('Fragments 2:\n{}\n'.format(qwc_list[1]))
print('Number of fragments: {}'.format(len(qwc_list)))

Fragments 1: 
-0.32760818967480887 [] +
0.04919764587136755 [X0 Y1 Y2 X3]

Fragments 2:
-0.04919764587136755 [X0 X1 Y2 Y3]

Number of fragments: 5


In [90]:
comm_groups = get_commuting_group(H)
print('Number of mutually commuting fragments: {}'.format(len(comm_groups)))
print('The first commuting group')
print(comm_groups[1])
print(comm_groups[2])

Number of mutually commuting fragments: 2
The first commuting group
-0.32760818967480887 [] +
-0.04919764587136755 [X0 X1 Y2 Y3] +
0.04919764587136755 [X0 Y1 Y2 X3] +
0.04919764587136755 [Y0 X1 X2 Y3] +
-0.04919764587136755 [Y0 Y1 X2 X3] +
0.15660062488237947 [Z0 Z1] +
0.10622904490856075 [Z0 Z2] +
0.15542669077992832 [Z0 Z3] +
0.15542669077992832 [Z1 Z2] +
0.10622904490856075 [Z1 Z3] +
0.16326768673564346 [Z2 Z3]
0.13716572937099497 [Z0] +
0.13716572937099492 [Z1] +
-0.13036292057109117 [Z2] +
-0.13036292057109117 [Z3]


In [96]:
%%time
#One has to loop over all groups

for i in range(1,len(comm_groups)+1):
    uqwc = get_qwc_unitary(comm_groups[i])
    qwc = remove_complex(uqwc * comm_groups[i] * uqwc)

    uz = get_zform_unitary(qwc)
    print("Checking whether U * U^+ is identity: {}".format(uz * uz))

    allz = remove_complex(uz * qwc * uz)
    print("\nThe all-z form of qwc fragment:\n{}".format(allz))

Checking whether U * U^+ is identity: 0.9999999999999998 []

The all-z form of qwc fragment:
-0.3276081896748086 [] +
0.15542669077992813 [Z0] +
0.15660062488237922 [Z0 Z1] +
0.049197645871367504 [Z0 Z1 Z3] +
0.10622904490856065 [Z0 Z2] +
-0.049197645871367504 [Z0 Z3] +
0.10622904490856065 [Z1] +
0.15542669077992813 [Z1 Z2] +
-0.049197645871367504 [Z1 Z2 Z3] +
0.16326768673564326 [Z2] +
0.049197645871367504 [Z2 Z3]
Checking whether U * U^+ is identity: 0.9999999999999996 []

The all-z form of qwc fragment:
0.13716572937099475 [Z0] +
0.13716572937099467 [Z1] +
-0.13036292057109095 [Z2] +
-0.13036292057109095 [Z3]
CPU times: user 75.6 ms, sys: 30 µs, total: 75.6 ms
Wall time: 74.6 ms


In [94]:
len(comm_groups)

2

In [61]:
#This is E0 at R=1 for h2 using H_eff
-0.531051349433764-0.5350572998841723

-1.0661086493179361